# HPC Fuzzing Pipeline - WORKING VERSION

This notebook has been fixed to work with the correct SWE-bench Docker images.

## Key Fixes:
- Uses correct image pattern: `swebench/sweb.eval.x86_64.{full_repo}_1776_{instance_id}:latest`
- SWE-bench images are PUBLIC - no Docker Hub authentication required!
- Apptainer environment variables properly configured

## Pipeline Overview:

### STATIC ANALYSIS (Host)
1. Patch Loading
2. Repository Setup  
3. Static Analysis (Pylint, Flake8, Radon, Mypy, Bandit)

### DYNAMIC ANALYSIS (Container)
4. Build Instance-Specific Singularity Container
5. Install Dependencies
6. Run Existing Tests
7. Patch Analysis
8. Generate Hypothesis Tests
9. Execute Tests
10. Coverage Analysis
11. Final Verdict

## Imports

In [2]:
import sys, subprocess, os
from pathlib import Path
import json, time, ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

PROJECT_ROOT = Path.cwd()
sys.path.append(str(PROJECT_ROOT))

# Dataset and patch handling
from swebench_integration import DatasetLoader, PatchLoader

# Dynamic container building system
from swebench_singularity import Config, SingularityBuilder, DockerImageResolver

# Analysis modules
from verifier.dynamic_analyzers.patch_analyzer import PatchAnalyzer
from verifier.dynamic_analyzers.test_generator import HypothesisTestGenerator
from verifier.dynamic_analyzers.singularity_executor import SingularityTestExecutor
from verifier.dynamic_analyzers.coverage_analyzer import CoverageAnalyzer
from verifier.dynamic_analyzers.test_patch_singularity import (
    install_package_in_singularity, 
    run_tests_in_singularity
)

# Static analysis
import streamlit.modules.static_eval.static_modules.code_quality as code_quality
import streamlit.modules.static_eval.static_modules.syntax_structure as syntax_structure
from verifier.utils.diff_utils import parse_unified_diff, filter_paths_to_py

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
print("✓ Imports OK")

/fs/nexus-scratch/ihbas/miniconda3/envs/verifier_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Imports OK


## Configuration Setup

**IMPORTANT**: This cell sets up the correct Docker image patterns for SWE-bench.

In [3]:
# Set Apptainer/Singularity environment variables
# NOTE: SWE-bench images are PUBLIC - authentication not required!
# Setting credentials anyway for compatibility with other private images
os.environ["APPTAINER_DOCKER_USERNAME"] = "nacheitor12"
os.environ["APPTAINER_DOCKER_PASSWORD"] = "wN/^4Me%,!5zz_q"
os.environ["SINGULARITY_DOCKER_USERNAME"] = "nacheitor12"
os.environ["SINGULARITY_DOCKER_PASSWORD"] = "wN/^4Me%,!5zz_q"

print("✓ Docker credentials set (not required for SWE-bench public images)")
print()

# Initialize configuration
config = Config()

# HPC cluster paths
config.set("singularity.cache_dir", "/fs/nexus-scratch/ihbas/.cache/swebench_singularity")
config.set("singularity.tmp_dir", "/fs/nexus-scratch/ihbas/.tmp/singularity_build")
config.set("singularity.cache_internal_dir", "/fs/nexus-scratch/ihbas/.singularity/cache")
config.set("singularity.build_timeout", 1800)  # 30 minutes
config.set("docker.max_retries", 3)

# CRITICAL: Use correct SWE-bench image pattern
# Format: swebench/sweb.eval.x86_64.{repo}_1776_{repo}-{version}:latest
# Example: swebench/sweb.eval.x86_64.scikit-learn_1776_scikit-learn-10297:latest
# NOTE: Uses {repo}-{version}, NOT {instance_id} which includes org prefix
config.set("docker.image_patterns", [
    "swebench/sweb.eval.x86_64.{repo}_1776_{repo}-{version}:latest",
])

# Initialize builder and resolver
builder = SingularityBuilder(config)
resolver = DockerImageResolver(config)

print("✓ Configuration initialized")
print(f"  Cache: {config.singularity_cache_dir}")
print(f"  Tmp: {config.singularity_tmp_dir}")
print(f"  Image pattern: {config.docker_image_patterns[0]}")
print(f"  Singularity: {builder.check_singularity_available()}")
print(f"  Docker: {builder.check_docker_available()}")

✓ Docker credentials set (not required for SWE-bench public images)

✓ Configuration initialized
  Cache: /fs/nexus-scratch/ihbas/.cache/swebench_singularity
  Tmp: /fs/nexus-scratch/ihbas/.tmp/singularity_build
  Image pattern: swebench/sweb.eval.x86_64.{repo}_1776_{repo}-{version}:latest


2025-11-19 15:32:23,276 - swebench_singularity.singularity_builder - INFO - Singularity available: apptainer version 1.4.4-1.el8


  Singularity: True
  Docker: False


---
# STATIC ANALYSIS
---

## Stage 1: Load Patch

In [4]:
REPO_FILTER = "scikit-learn/scikit-learn"

loader = DatasetLoader("princeton-nlp/SWE-bench_Verified", hf_mode=True, split="test")
sample = next(loader.iter_samples(limit=1, filter_repo=REPO_FILTER), None)

if sample:
    instance_id = sample.get('metadata', {}).get('instance_id', 'unknown')
    print(f"✓ {instance_id}")
    print(f"  Repo: {sample['repo']}")
    print(f"\nPatch preview:\n{sample['patch'][:400]}...")
else:
    raise Exception(f"No sample found for {REPO_FILTER}")

✓ scikit-learn__scikit-learn-10297
  Repo: scikit-learn/scikit-learn

Patch preview:
diff --git a/sklearn/linear_model/ridge.py b/sklearn/linear_model/ridge.py
--- a/sklearn/linear_model/ridge.py
+++ b/sklearn/linear_model/ridge.py
@@ -1212,18 +1212,18 @@ class RidgeCV(_BaseRidgeCV, RegressorMixin):
 
     store_cv_values : boolean, default=False
         Flag indicating if the cross-validation values corresponding to
-        each alpha should be stored in the `cv_values_` attrib...


## Stage 2: Setup Repository

In [5]:
patcher = PatchLoader(sample=sample, repos_root="./repos_temp")
repo_path = patcher.clone_repository()
patch_result = patcher.apply_patch()

print(f"✓ Repo: {repo_path}")
print(f"✓ Patch: {'Applied' if patch_result['applied'] else 'FAILED'}")

[+] Cloning scikit-learn/scikit-learn into /fs/nexus-scratch/ihbas/verifier_harness/repos_temp/scikit-learn__scikit-learn ...
✓ Repo: /fs/nexus-scratch/ihbas/verifier_harness/repos_temp/scikit-learn__scikit-learn
✓ Patch: Applied


## Stage 2b: Apply Test Patch

In [6]:
test_patch = sample.get('metadata', {}).get('test_patch', '')

if test_patch and test_patch.strip():
    print("📝 Applying test_patch...")
    try:
        test_patch_result = patcher.apply_additional_patch(test_patch)
        print(f"✓ Test patch applied: {test_patch_result.get('log', 'success')}")
    except Exception as e:
        print(f"⚠️ Test patch application failed: {e}")
else:
    print("ℹ️  No test_patch in metadata")

📝 Applying test_patch...
✓ Test patch applied: Additional patch applied successfully.


## Stage 3: Static Analysis

In [7]:
static_config = {
    'checks': {'pylint': True, 'flake8': True, 'radon': True, 'mypy': True, 'bandit': True},
    'weights': {'pylint': 0.5, 'flake8': 0.15, 'radon': 0.25, 'mypy': 0.05, 'bandit': 0.05}
}

print("🔍 Static analysis...")
cq_results = code_quality.analyze(str(repo_path), sample['patch'], static_config)
ss_results = syntax_structure.run_syntax_structure_analysis(str(repo_path), sample['patch'])

sqi_data = cq_results.get('sqi', {})
print(f"✓ SQI: {sqi_data.get('SQI', 0)}/100 ({sqi_data.get('classification', 'Unknown')})")

🔍 Static analysis...


✓ SQI: 61.54/100 (Fair)


---
# DYNAMIC ANALYSIS (Container)
---

## Stage 4: Build Container

This will download and convert the SWE-bench Docker image to Singularity format.
The images are PUBLIC and don't require authentication.

In [8]:
print(f"🐳 Building container for {instance_id}...\n")

# Check what Docker image will be used
docker_image = resolver.find_available_image(instance_id, check_existence=False)
if docker_image:
    print(f"  Docker image: {docker_image.full_name}")
    print(f"  Registry: {docker_image.registry}")
    print(f"  Tag: {docker_image.tag}\n")

# Build the container (will use cache if available)
build_result = builder.build_instance(
    instance_id=instance_id,
    force_rebuild=False,
    check_docker_exists=False
)

if build_result.success:
    CONTAINER_IMAGE_PATH = build_result.sif_path
    cache_status = "(from cache)" if build_result.from_cache else "(newly built)"
    print(f"✓ Container ready {cache_status}")
    print(f"  Path: {CONTAINER_IMAGE_PATH}")
    print(f"  Build time: {build_result.build_time_seconds:.1f}s")
    
    # Verify container works
    result = subprocess.run(
        ["singularity", "exec", str(CONTAINER_IMAGE_PATH), "python", "--version"],
        capture_output=True, text=True
    )
    print(f"  Python: {result.stdout.strip()}")
else:
    print(f"❌ Container build failed: {build_result.error_message}")
    raise Exception("Cannot proceed without container")

2025-11-19 15:34:02,694 - swebench_singularity.singularity_builder - INFO - Using cached image for scikit-learn__scikit-learn-10297: /fs/nexus-scratch/ihbas/.cache/swebench_singularity/scikit-learn/scikit-learn__scikit-learn-10297.sif


🐳 Building container for scikit-learn__scikit-learn-10297...

  Docker image: docker.io/swebench/sweb.eval.x86_64.scikit-learn_1776_scikit-learn-10297:latest
  Registry: docker.io
  Tag: latest

✓ Container ready (from cache)
  Path: /fs/nexus-scratch/ihbas/.cache/swebench_singularity/scikit-learn/scikit-learn__scikit-learn-10297.sif
  Build time: 0.0s
  Python: Python 3.11.5


## Stage 5: Install Dependencies

In [9]:
print("📦 Installing dependencies...\n")

install_result = install_package_in_singularity(
    repo_path=Path(repo_path),
    image_path=str(CONTAINER_IMAGE_PATH)
)

if install_result.get("installed"):
    print("✓ Dependencies installed")
elif install_result.get("returncode") != 0:
    print(f"⚠️ Install issues (code {install_result.get('returncode')})")
    print(install_result.get('stderr', '')[-500:])
else:
    print("⚠️ No setup.py/pyproject.toml (will use PYTHONPATH mode)")

📦 Installing dependencies...

📦 Installing package and dependencies in: /fs/nexus-scratch/ihbas/verifier_harness/repos_temp/scikit-learn__scikit-learn
   Setup files found: setup.py=True, pyproject.toml=False, setup.cfg=True
   Attempting editable install...
   Installing build dependencies...
   Configuring git for build...
   Fetching git tags for version detection...
   Building C extensions in-place...
⚠️  In-place build failed, trying editable install...
ℹ️  Build and install failed (will use PYTHONPATH mode)
   Note: This may not work for packages with C extensions
⚠️ No setup.py/pyproject.toml (will use PYTHONPATH mode)


## Stage 6: Run Existing Tests

In [10]:
print("🧪 Running existing tests...\n")

fail_to_pass = sample.get('metadata', {}).get('FAIL_TO_PASS', '[]')
pass_to_pass = sample.get('metadata', {}).get('PASS_TO_PASS', '[]')

print(f"  FAIL_TO_PASS: {fail_to_pass}")
print(f"  PASS_TO_PASS: {pass_to_pass}\n")

try:
    f2p = ast.literal_eval(fail_to_pass) if isinstance(fail_to_pass, str) else fail_to_pass
    p2p = ast.literal_eval(pass_to_pass) if isinstance(pass_to_pass, str) else pass_to_pass
except:
    f2p, p2p = [], []

all_tests = f2p + p2p

test_result = run_tests_in_singularity(
    repo_path=Path(repo_path),
    tests=all_tests,
    image_path=str(CONTAINER_IMAGE_PATH)
)

print(f"Exit code: {test_result['returncode']}")
print((test_result['stdout'] + test_result['stderr'])[-1500:])
print(f"\n{'✓' if test_result['returncode'] == 0 else '⚠️'} Tests {'passed' if test_result['returncode'] == 0 else 'had issues'}")

🧪 Running existing tests...

  FAIL_TO_PASS: ["sklearn/linear_model/tests/test_ridge.py::test_ridge_classifier_cv_store_cv_values"]
  PASS_TO_PASS: ["sklearn/linear_model/tests/test_ridge.py::test_ridge", "sklearn/linear_model/tests/test_ridge.py::test_primal_dual_relationship", "sklearn/linear_model/tests/test_ridge.py::test_ridge_singular", "sklearn/linear_model/tests/test_ridge.py::test_ridge_regression_sample_weights", "sklearn/linear_model/tests/test_ridge.py::test_ridge_sample_weights", "sklearn/linear_model/tests/test_ridge.py::test_ridge_shapes", "sklearn/linear_model/tests/test_ridge.py::test_ridge_intercept", "sklearn/linear_model/tests/test_ridge.py::test_toy_ridge_object", "sklearn/linear_model/tests/test_ridge.py::test_ridge_vs_lstsq", "sklearn/linear_model/tests/test_ridge.py::test_ridge_individual_penalties", "sklearn/linear_model/tests/test_ridge.py::test_ridge_cv_sparse_svd", "sklearn/linear_model/tests/test_ridge.py::test_ridge_sparse_svd", "sklearn/linear_model/tests

✓ C extensions copied
🧪 Running tests in Singularity:
  singularity exec --bind /fs/nexus-scratch/ihbas/verifier_harness/repos_temp/scikit-learn__scikit-learn:/workspace --pwd /workspace --env PYTHONPATH=/workspace /fs/nexus-scratch/ihbas/.cache/swebench_singularity/scikit-learn/scikit-learn__scikit-learn-10297.sif /opt/miniconda3/envs/testbed/bin/python -m pytest -q sklearn/linear_model/tests/test_ridge.py::test_ridge_classifier_cv_store_cv_values sklearn/linear_model/tests/test_ridge.py::test_ridge sklearn/linear_model/tests/test_ridge.py::test_primal_dual_relationship sklearn/linear_model/tests/test_ridge.py::test_ridge_singular sklearn/linear_model/tests/test_ridge.py::test_ridge_regression_sample_weights sklearn/linear_model/tests/test_ridge.py::test_ridge_sample_weights sklearn/linear_model/tests/test_ridge.py::test_ridge_shapes sklearn/linear_model/tests/test_ridge.py::test_ridge_intercept sklearn/linear_model/tests/test_ridge.py::test_toy_ridge_object sklearn/linear_model/tests

## Remaining Stages

The notebook continues with:
- Stage 7: Patch Analysis
- Stage 8: Generate Hypothesis Tests
- Stage 9: Execute Tests
- Stage 10: Coverage Analysis
- Stage 11: Final Verdict

Add the remaining cells from your original notebook here as needed.